In [1]:
import cv2
import pandas as pd
import numpy as np
import json
import random

ModuleNotFoundError: No module named 'cv2'

In [2]:
with open('bboxes.json', 'r') as j:
     contents = json.loads(j.read())

In [3]:
def convert_to_yolo_box_params(box_coordinates, im_w, im_h):

        ans = []

        ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
        ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      
        ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
        ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
        return ans

In [3]:
def show_frames_w_bboxes(trim_name, df):
    vidcap = cv2.VideoCapture(f'Video/Локотех_{trim_name}.mp4')
    success, image = vidcap.read()
    count = 1
    
    while success:
        objects = df[df.frame == count]
        for _, obj in objects.iterrows():
            name = obj.label
        
            if name == 'bot':
                color = (0, 230, 255)
            elif name == 'mob':
                color = (188, 0, 255)
            else:
                color = (255, 255, 255)
                
            t = int(obj.top)
            h = int(obj.height)
            l = int(obj.left)
            w = int(obj.width)
            
            cv2.rectangle(image, (l + w, t), (l, t + h), color, 2)
            
        cv2.imshow('lol', image)
        cv2.waitKey(0)
        cv2.destroyWindow('lol')
        success, image = vidcap.read()
        count += 1

In [4]:
its = list(contents[2]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames'].items())

In [5]:
matrix = []
for frame in its:
    frame_num = frame[0]
    objects = frame[1]['objects']
    obj_id = objects.keys()

    for aidi in obj_id:
        row = []
        bbox = [k[1] for k in [*objects[aidi]['bounding_box'].items()]]
        row.append([frame_num, aidi, objects[aidi]['name'], *bbox])
        matrix.extend(row)

In [6]:
df = pd.DataFrame(matrix, columns=['frame', 'object_id', 'label', 'top', 'left', 'height', 'width'])
df.frame = df.frame.astype(int)

In [7]:
pd.options.mode.chained_assignment = None 

monster_df = pd.DataFrame()
for obj in df.object_id.unique().tolist():
    over_df = pd.DataFrame()
    slice_bob = df[df.object_id == obj].sort_values('frame')
    for first, second in zip(slice_bob.frame.unique().tolist(), slice_bob.frame.unique().tolist()[1:]):
        frame_numbers = range(first, second + 1)
        sub_set_first = slice_bob[slice_bob.frame == first][['top', 'left', 'height', 'width']]
        sub_set_second = slice_bob[slice_bob.frame == second][['top', 'left', 'height', 'width']]
        temp_df = pd.DataFrame({'frame': frame_numbers, 
                                'object_id': obj, 
                                'label': slice_bob.label.iloc[0], 
                                'top': sub_set_first.top.iloc[0], 
                                'left': sub_set_first.left.iloc[0], 
                                'height': sub_set_first.height.iloc[0], 
                                'width': sub_set_first.width.iloc[0]})
        temp_df['top'].iloc[1:] = None
        temp_df['top'].iloc[-1] = sub_set_second.top.iloc[0]
        temp_df['left'].iloc[1:] = None
        temp_df['left'].iloc[-1] = sub_set_second.left.iloc[0]
        temp_df['height'].iloc[1:] = None
        temp_df['height'].iloc[-1] = sub_set_second.height.iloc[0]
        temp_df['width'].iloc[1:] = None
        temp_df['width'].iloc[-1] = sub_set_second.width.iloc[0]
        temp_df = temp_df.interpolate()
        over_df = pd.concat([over_df, temp_df])
    monster_df = pd.concat([monster_df, over_df])
    
monster_df = monster_df.drop_duplicates().sort_values('frame')

In [9]:
show_frames_w_bboxes('Trim3', monster_df)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'lol' in function 'cvDestroyWindow'
